In [1]:
import pandas as pd
import numpy as np

In [2]:
files = [
    'final average/1229_blend_minimax.csv',
    'final average/1231_blend_minimax.csv',
    'final average/1237_blend_minimax.csv',
    'final average/1239_blend_minimax_median_se_dense.csv',
    'final average/1242_blend_minmax_best_8th_best.csv',
    'final average/1258_blend_minimax_median_corr_less_09.csv',
    'final average/1260_blend_minimax.csv',
]
file_num = len(files)

In [3]:
test = pd.read_json('Data/test.json', dtype={'inc_angle': str})
read_ids = pd.read_csv('real ids.csv')['id'].values
print(len(read_ids))

3425


In [7]:
th=6
leak_df = pd.read_csv('analysis submissions/data_leakage_th_{}.csv'.format(th))
leak_angles = leak_df.inc_angle.values
leak_ice = leak_df.is_iceberg.values

In [10]:
def update_patch(f):
    df = pd.read_csv(f)
    answers = df['is_iceberg'].values

    for i in range(test.shape[0]):
        if test.id.values[i] in read_ids and test.inc_angle.values[i] in leak_angles:

            ix, = np.where( leak_angles==test.inc_angle.values[i] )
            new_ans = leak_ice[ix]
            new_ans = max(0.0001, min(new_ans, 0.9999))

            answers[i] = new_ans
            

    df['is_iceberg'] = answers
    df.to_csv(f.replace('.csv', '_th{}.csv'.format(th)), index=False, float_format="%.15f")

In [11]:
for f in files:
    update_patch(f)